In [77]:
import spotipy
import pandas as pd
from pprint import pprint

from auth.spotify_auth import authenticate_user

sp = authenticate_user()

In [121]:
def get_top_tracks(sp, time_range="medium_term", limit=50):

    # fetach top tracks
    results = sp.current_user_top_tracks(time_range=time_range, limit=limit)
    top_tracks = results['items']

    DATE_FORMAT = "%Y-%m-%d"

    # flattens nested json to dataframe
    df = pd.json_normalize(top_tracks)

    # columns to use
    df = df[[
        "id",
        "name",
        "album.name",
        "album.release_date",
        "duration_ms",
        "popularity",
        "explicit",
        "track_number",
        "artists",
        "external_urls.spotify",
    ]]

    df.rename(columns={
        "id": "track_id",
        "name": "track_name",
        "album.name": "album_name",
        "album.release_date": "album_release_date",
        "external_urls.spotify": "spotify_url",
    }, inplace=True)

    # #create track l"inks to spotify
    # df["spotify_url"] = df["track_id"].apply(lambda tid: f"https://open.spotify.com/track/{tid}")

    #collapse the list of artists into a single comma separated string
    df["artist_names"] = df["artists"].apply(
        lambda artists: ", ".join([artist["name"] for artist in artists]) #for each artist in artists list join the names with a comma
    )

    # convert duration from ms to minutes
    df["duration_min"] = df["duration_ms"] / 60000

    # reorder and drop columns

    df = df[[
        "track_id",
        "track_name",
        "track_number",
        "duration_min",
        "album_name",
        "artist_names",
        "album_release_date",
        "popularity",
        "explicit",
        "spotify_url",
    ]]

    df["album_release_date"] = pd.to_datetime(df["album_release_date"], format=DATE_FORMAT)


    return df

In [122]:
df = get_top_tracks(sp, time_range="medium_term", limit=3)
df

,track_id,track_name,track_number,duration_min,album_name,artist_names,album_release_date,popularity,explicit,spotify_url
0,5Nz7hI3cCOHmMR4vSLJ1An,GIMME A HUG,10,3.217567,$ome $exy $ongs 4 U,Drake,2025-02-14,80,True,https://open.spotify.com/track/5Nz7hI3cCOHmMR4...
1,2u9S9JJ6hTZS3Vf22HOZKg,NOKIA,14,4.017050,$ome $exy $ongs 4 U,Drake,2025-02-14,94,True,https://open.spotify.com/track/2u9S9JJ6hTZS3Vf...
2,3FHO87BSOEVAsedSnCTnvz,BRIAN STEEL,9,1.856767,$ome $exy $ongs 4 U,Drake,2025-02-14,72,True,https://open.spotify.com/track/3FHO87BSOEVAsed...
